In [1]:
from modules.utils import load_datasets_statforecast_uni

In [2]:
data_path = '/Users/moji/PyTSF-MfG/data'
datasets = load_datasets_statforecast_uni(data_path)


Loading dataset: air_passengers
File path: /Users/moji/PyTSF-MfG/data/AirPassengers.csv
Date parsing complete for air_passengers
Dataset shape for air_passengers: (144, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   unique_id  144 non-null    object        
 1   ds         144 non-null    datetime64[ns]
 2   y          144 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 3.5+ KB
None
                   unique_id         ds    y
0  air_passengers_Passengers 1949-01-01  112
1  air_passengers_Passengers 1949-02-01  118
2  air_passengers_Passengers 1949-03-01  132
3  air_passengers_Passengers 1949-04-01  129
4  air_passengers_Passengers 1949-05-01  121
Most common time difference: 31 days 00:00:00

Loading dataset: ETTh1
File path: /Users/moji/PyTSF-MfG/data/ETTh1.csv
Date parsing complete for 

In [ ]:
datasets['ETTh1']

In [ ]:
hi